In [504]:
import numpy as np
import torch

In [505]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [506]:
x_data.shape, x_data.dtype, x_data.device

(torch.Size([2, 2]), torch.int64, device(type='cpu'))

In [507]:
x_data @ torch.ones_like(x_data)
x_data * torch.ones_like(x_data)
x_data @ torch.eye(2,2, dtype=torch.long)
x_data * torch.eye(2,2, dtype=torch.long)

tensor([[1, 0],
        [0, 4]])

In [508]:
x_data.dtype

torch.int64

In [509]:
x_data.numpy()

array([[1, 2],
       [3, 4]])

In [510]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# pyG Dataset

In [511]:
import numpy as np
import networkx as nx
import torch

In [512]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx, index_to_mask, mask_to_index

In [513]:
G = nx.DiGraph()
G.add_edges_from([
    (0, 2),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (4, 6)
])

In [514]:
torch.tensor([2])
index_to_mask(torch.tensor([2]))

tensor([False, False,  True])

In [515]:
d = from_networkx(G)
d.train_mask = index_to_mask(torch.tensor([2]), size=d.num_nodes)
d2 = from_networkx(G)
d2.train_mask = index_to_mask(torch.tensor([3]), size=d.num_nodes)

In [516]:
loader = DataLoader([d, d2, d, d2], batch_size=3, shuffle=True)

In [517]:
for data in loader:
    print(data.train_mask)

tensor([False, False, False,  True, False, False, False, False, False,  True,
        False, False, False, False, False, False, False,  True, False, False,
        False])
tensor([False, False,  True, False, False, False, False])


In [518]:
out = torch.rand(7)
out[data.train_mask]

tensor([0.9403])

In [519]:
d.node_stores

[{'edge_index': tensor([[0, 1, 2, 3, 4, 4],
         [1, 3, 1, 4, 5, 6]]), 'num_nodes': 7, 'train_mask': tensor([False, False,  True, False, False, False, False])}]

In [520]:
from torch_geometric.utils import to_networkx
G2 = to_networkx(d)
# nx.draw_networkx(G2)

In [521]:
# nx.draw_networkx(G)

In [522]:
G = nx.DiGraph()
G.add_edges_from([
    ('A', 'C'),
    ('B', 'C'),
    ('C', 'D'),
    ('D', 'E'),
    ('D', 'F'),
])

attr = {node:[float(i), float(i+1)] for i,node in enumerate(G.nodes)}
nx.set_node_attributes(G, attr, name='toto')
G = nx.convert_node_labels_to_integers(G, label_attribute='name')

In [523]:
attr

{'A': [0.0, 1.0],
 'C': [1.0, 2.0],
 'B': [2.0, 3.0],
 'D': [3.0, 4.0],
 'E': [4.0, 5.0],
 'F': [5.0, 6.0]}

In [524]:
d = from_networkx(G.reverse(), group_node_attrs=['toto'])
d.train_mask = index_to_mask(torch.tensor(d.name.index('C')), size=d.num_nodes)
d.y = torch.tensor([[0., 10.]]*d.num_nodes)
d, d.node_stores

(Data(edge_index=[2, 5], name=[6], x=[6, 2], train_mask=[6], y=[6, 2]),
 [{'edge_index': tensor([[1, 1, 3, 4, 5],
          [0, 2, 1, 3, 3]]), 'name': ['A', 'C', 'B', 'D', 'E', 'F'], 'x': tensor([[0., 1.],
          [1., 2.],
          [2., 3.],
          [3., 4.],
          [4., 5.],
          [5., 6.]]), 'train_mask': tensor([False,  True, False, False, False, False]), 'y': tensor([[ 0., 10.],
          [ 0., 10.],
          [ 0., 10.],
          [ 0., 10.],
          [ 0., 10.],
          [ 0., 10.]])}])

In [525]:
d.num_node_features, d.x, d.x.dtype, d.validate(), d.y

(2,
 tensor([[0., 1.],
         [1., 2.],
         [2., 3.],
         [3., 4.],
         [4., 5.],
         [5., 6.]]),
 torch.float32,
 True,
 tensor([[ 0., 10.],
         [ 0., 10.],
         [ 0., 10.],
         [ 0., 10.],
         [ 0., 10.],
         [ 0., 10.]]))

In [526]:
d2 = Data.clone(d)
d2.x += 1
d2.train_mask = index_to_mask(torch.tensor(2), size=d2.num_nodes)

In [527]:
loader = DataLoader([d, d2]*3, batch_size=4)

In [528]:
for batch in loader:
    display(batch)

DataBatch(edge_index=[2, 20], name=[4], x=[24, 2], train_mask=[24], y=[24, 2], batch=[24], ptr=[5])

DataBatch(edge_index=[2, 10], name=[2], x=[12, 2], train_mask=[12], y=[12, 2], batch=[12], ptr=[3])

In [545]:
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv

class DQGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(d.num_node_features, 16)
        self.conv2 = GCNConv(16, 8)
        self.linear = Linear(8, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.linear(x)

        return x

In [546]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


device(type='cpu')

In [535]:
from torch.nn import SmoothL1Loss, MSELoss

In [567]:
model = DQGCN().to(device)
model(d)[d.train_mask], model(d2)[d2.train_mask]

(tensor([[ 0.3873, -0.4448]], grad_fn=<IndexBackward0>),
 tensor([[ 0.3149, -0.4063]], grad_fn=<IndexBackward0>))

In [568]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [569]:
loss_function = SmoothL1Loss()

model.train()
for epoch in range(100):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)[batch.train_mask].gather(1, torch.tensor([[0], [1]]))
        target = batch.y[batch.train_mask].gather(1, torch.tensor([[0], [1]]))

        loss = loss_function(target, out)
        loss.backward()
        optimizer.step()


In [570]:
model.eval()
model(d)[d.train_mask], model(d2)[d2.train_mask]


(tensor([[-3.8341e-05,  1.2375e+01]], grad_fn=<IndexBackward0>),
 tensor([[-0.0428, 10.0007]], grad_fn=<IndexBackward0>))

In [544]:
d2.x

tensor([[1., 2.],
        [2., 3.],
        [3., 4.],
        [4., 5.],
        [5., 6.],
        [6., 7.]])